# Analysis of the results of the NNI+DeCoSTAR experiments of 2018_07_15-16
Cedric Chauve, July 2018, 19

In [1]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Overview

This notebook contains an analysis of the results of the experiments consisting into 
* simulating the evolution of genomes (gene trees and gene orders) using <a href="https://www.biorxiv.org/content/early/2018/06/07/339473">Zombi</a>;
* adding noise by performing NNI tree rearrangements, with various levels of noise;
* reconstructing ancestral adjacencies using <a href="https://doi.org/10.1093/gbe/evx069">DeCoSTAR</a>;
* recording statistics about the scores of reconciliation, DeCoSTAR and linearity of ancestral gene orders
in order to assess if the linearity of ancestral gene orders can be used as a diagnostic measure of correctness of the gene trees.

The experiments were run on two datasets of simulated gene trees, one simulated with no HGT and one simulated with HGT, respectively called the noHGT and HGT datasets.

## Data and results

The data and results of the experiments for the noHGT dataset are available in the directory ../../exp/2018_07_15_D10T0L20I10T10_NNI_DECOSTAR. The data and results of the experiments for the noHGT dataset are available in the directory ../../exp/2018_07_16_D10T10L20I10T10_NNI_DECOSTAR.

**TO DO**: describe in more detail the experiments.

In each of these directory, a summary file of the results is available in the file results/summary_1.

## Analysis of the results with the noHGT dataset.

In [3]:
# Reading summary file (path to file, F) and creating a dataframe

# Correspondance between dataset names and lambda parameter of the Poisson law used to determine the average number of NNI per gene tree
LAMBDA={'lambda_025':0.25,'lambda_05':0.5,'lambda_1':1,'lambda_2':2,'lambda_3':3,'lambda_5':5,'lambda_7':7,'lambda_10':10,'lambda_20':20,'lambda_30':30,'lambda_50'
:50}

def read_summary_file(F):
    SUMMARY_FILE = open(F,'r').readlines()
    SUMMARY_AUX  = {}
    for l in SUMMARY_FILE:
        if l[0]!="#":
            l1 = l.rstrip().split()
            (name,plambda,rf,gdup,gloss,hgt,rec,again,abreak,deco,linearity) = (l1[0],float(LAMBDA[l1[0]]),float(l1[1]),float(l1[2]),float(l1[3]),float(l1[4]),float(l1[5]),float(l1[6]),float(l1[7]),float(l1[8]),float(l1[9]))
            SUMMARY_AUX[name] = (plambda,rf,gdup,gloss,hgt,rec,again,abreak,deco,linearity)  
    SUMMARY = pd.DataFrame.transpose(pd.DataFrame(SUMMARY_AUX))
    SUMMARY.columns=['Lambda','RF','nb_dup','nb_loss','nb_hgt','rec_score','nb_gain','nb_break','DeCo_score','linearity_score']
    return(SUMMARY)

In [4]:
NOHGT_SUMMARY=read_summary_file('../../exp/2018_07_15_D10T0L20I10T10_NNI_DECOSTAR/results/summary_1')

In [5]:
NOHGT_SUMMARY

,Lambda,RF,nb_dup,nb_loss,nb_hgt,rec_score,nb_gain,nb_break,DeCo_score,linearity_score
lambda_025,0.25,0.13,183.0,528.0,0.0,894.0,1782.0,294.0,5640.0,592.0
lambda_05,0.50,0.22,273.0,815.0,0.0,1361.0,1961.0,287.0,6170.0,980.0
lambda_1,1.00,0.47,503.0,1537.0,0.0,2543.0,2399.0,279.0,7476.0,1956.0
lambda_10,10.00,3.97,3599.0,11841.0,0.0,19039.0,7467.0,140.0,22541.0,13336.0
lambda_2,2.00,0.96,974.0,3027.0,0.0,4975.0,3259.0,249.0,10026.0,3834.0
lambda_20,20.00,6.88,5923.0,20537.0,0.0,32383.0,10404.0,111.0,31323.0,21152.0
lambda_3,3.00,1.36,1361.0,4238.0,0.0,6960.0,3932.0,232.0,12028.0,5234.0
lambda_30,30.00,8.88,7235.0,26320.0,0.0,40790.0,12122.0,88.0,36454.0,26198.0
lambda_5,5.00,2.16,2097.0,6643.0,0.0,10837.0,5206.0,207.0,15825.0,8014.0
lambda_50,50.00,11.90,9119.0,35959.0,0.0,54197.0,14232.0,53.0,42749.0,34026.0


In [6]:
# RF distance versus Lambda
plt.scatter(NOHGT_SUMMARY['Lambda'], NOHGT_SUMMARY['RF'])
plt.xlabel("Lambda")
plt.ylabel("RF distance")
plt.savefig('2018_07_15_D10T0L20I10T10_NNI_DECOSTAR_RF.png') 
plt.close()

In [7]:
# Plotting the three scores
plt.scatter(NOHGT_SUMMARY['RF'], NOHGT_SUMMARY['rec_score'], c='g', marker='o')
plt.scatter(NOHGT_SUMMARY['RF'], NOHGT_SUMMARY['DeCo_score'], c='b', marker='x')
plt.scatter(NOHGT_SUMMARY['RF'], NOHGT_SUMMARY['linearity_score'], c='r', marker='+')
plt.xlabel("RF distance")
plt.ylabel("Scores")
plt.legend(['reconciliation','DeCoSTAR','linearity'],loc=4)
plt.savefig('2018_07_15_D10T0L20I10T10_NNI_DECOSTAR_scores.png') 
plt.close()

plt.scatter(NOHGT_SUMMARY['RF'], NOHGT_SUMMARY['rec_score'], c='g', marker='o')
plt.scatter(NOHGT_SUMMARY['RF'], NOHGT_SUMMARY['DeCo_score'], c='b', marker='x')
plt.xlabel("RF distance")
plt.ylabel("Scores")
plt.legend(['reconciliation','DeCoSTAR'],loc=4)
plt.savefig('2018_07_15_D10T0L20I10T10_NNI_DECOSTAR_scores_rec_DeCo.png') 
plt.close()

plt.scatter(NOHGT_SUMMARY['RF'], NOHGT_SUMMARY['linearity_score'], c='r', marker='+')
plt.xlabel("RF distance")
plt.ylabel("Linearity score")
plt.savefig('2018_07_15_D10T0L20I10T10_NNI_DECOSTAR_linearity.png') 
plt.close()

<img src="2018_07_15_D10T0L20I10T10_NNI_DECOSTAR_RF.png">
<img src="2018_07_15_D10T0L20I10T10_NNI_DECOSTAR_scores.png">
<img src="2018_07_15_D10T0L20I10T10_NNI_DECOSTAR_scores_rec_DeCo.png">
<img src="2018_07_15_D10T0L20I10T10_NNI_DECOSTAR_linearity.png">

## Analysis of the results with the HGT dataset.

In [8]:
HGT_SUMMARY=read_summary_file('../../exp/2018_07_16_D10T10L20I10T10_NNI_DECOSTAR/results/summary_1')

In [9]:
HGT_SUMMARY

,Lambda,RF,nb_dup,nb_loss,nb_hgt,rec_score,nb_gain,nb_break,DeCo_score,linearity_score
lambda_05,0.5,0.25,121.0,422.0,314.0,1606.0,2143.0,701.0,7130.0,182.0
lambda_1,1.0,0.45,120.0,599.0,513.0,2378.0,2530.0,875.0,8465.0,272.0
lambda_10,10.0,3.96,103.0,2925.0,3468.0,13535.0,7906.0,2586.0,26304.0,2420.0
lambda_2,2.0,0.93,115.0,1009.0,967.0,4140.0,3381.0,1203.0,11346.0,482.0
lambda_20,20.0,6.65,112.0,4111.0,5276.0,20163.0,10960.0,2785.0,35665.0,4700.0
lambda_3,3.0,1.38,112.0,1356.0,1377.0,5711.0,4139.0,1558.0,13975.0,682.0
lambda_30,30.0,8.93,130.0,4893.0,6637.0,25064.0,13173.0,2838.0,42357.0,6790.0
lambda_5,5.0,2.18,115.0,1867.0,2021.0,8160.0,5323.0,1905.0,17874.0,1084.0
lambda_50,50.0,11.88,141.0,5812.0,8140.0,30514.0,15542.0,2824.0,49450.0,9298.0
lambda_7,7.0,2.97,111.0,2359.0,2690.0,10651.0,6511.0,2302.0,21835.0,1636.0


In [10]:
# RF distance versus Lambda
plt.scatter(HGT_SUMMARY['Lambda'], HGT_SUMMARY['RF'])
plt.xlabel("Lambda")
plt.ylabel("RF distance")
plt.savefig('2018_07_16_D10T10L20I10T10_NNI_DECOSTAR_RF.png') 
plt.close()

# Plotting the three scores
# Plotting the three scores
plt.scatter(HGT_SUMMARY['RF'], HGT_SUMMARY['rec_score'], c='g', marker='o')
plt.scatter(HGT_SUMMARY['RF'], HGT_SUMMARY['DeCo_score'], c='b', marker='x')
plt.scatter(HGT_SUMMARY['RF'], HGT_SUMMARY['linearity_score'], c='r', marker='+')
plt.xlabel("RF distance")
plt.ylabel("Scores")
plt.legend(['reconciliation','DeCoSTAR','linearity'],loc=4)
plt.savefig('2018_07_16_D10T10L20I10T10_NNI_DECOSTAR_scores.png') 
plt.close()

plt.scatter(HGT_SUMMARY['RF'], HGT_SUMMARY['rec_score'], c='g', marker='o')
plt.scatter(HGT_SUMMARY['RF'], HGT_SUMMARY['DeCo_score'], c='b', marker='x')
plt.xlabel("RF distance")
plt.ylabel("Scores")
plt.legend(['reconciliation','DeCoSTAR'],loc=4)
plt.savefig('2018_07_16_D10T10L20I10T10_NNI_DECOSTAR_scores_rec_DeCo.png') 
plt.close()

plt.scatter(HGT_SUMMARY['RF'], HGT_SUMMARY['linearity_score'], c='r', marker='+')
plt.xlabel("RF distance")
plt.ylabel("Linearity score")
plt.savefig('2018_07_16_D10T10L20I10T10_NNI_DECOSTAR_linearity.png') 
plt.close()

<img src="2018_07_16_D10T10L20I10T10_NNI_DECOSTAR_RF.png">
<img src="2018_07_16_D10T10L20I10T10_NNI_DECOSTAR_scores.png">
<img src="2018_07_16_D10T10L20I10T10_NNI_DECOSTAR_scores_rec_DeCo.png">
<img src="2018_07_16_D10T10L20I10T10_NNI_DECOSTAR_linearity.png">